<a href="https://www.kaggle.com/code/singhayush16/spaceship-titanic?scriptVersionId=140314620" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [128]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# **Training Part**

In [129]:
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [130]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [131]:
train.Age.interpolate(inplace=True)

In [132]:
train.RoomService.interpolate(inplace=True)

In [133]:
train.FoodCourt.interpolate(inplace=True)
train.ShoppingMall.interpolate(inplace=True)
train.Spa.interpolate(inplace=True)
train.VRDeck.interpolate(inplace=True)

In [134]:
train.shape

(8693, 14)

In [135]:
train.CryoSleep.fillna(True,inplace=True)

In [136]:
train.HomePlanet.fillna('Mars',inplace=True)

In [137]:
train[['deck', 'num', 'side']]=train.Cabin.str.split('/',expand=True)

In [138]:
train.deck.fillna('T',inplace=True)

In [139]:
train.num=train.num.astype(float)

In [140]:
train.num=train.num.interpolate(method='linear',limit_direction='both').round().astype(int)

In [141]:
train.side.fillna('P',inplace=True)

In [142]:
train.VIP.fillna(True,inplace=True)

In [143]:
train.Destination.fillna('PSO J318.5-22',inplace=True)

In [144]:
train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           199
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
deck              0
num               0
side              0
dtype: int64

In [145]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,deck,num,side
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [146]:
df=train.drop(['PassengerId','Name','Cabin'],axis=1)
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S


In [147]:
df.HomePlanet.replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)

In [148]:
df.replace({'side':{'S':0,'P':1}},inplace=True)

In [149]:
df.replace({'deck':{'F':0,'G':1,'E':2,'B':2,'C':2,'D':2,'A':2,'T':2}},inplace=True)

In [150]:
df.replace({'VIP':{False:0,True:1}},inplace=True)

In [151]:
df.replace({'Destination':{'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':1}},inplace=True)

In [152]:
df.replace({'CryoSleep':{False:0,True:1}},inplace=True)

In [153]:
df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,deck,num,side
0,1,0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,False,2,0,1
1,0,0,0,24.0,0,109.0,9.0,25.0,549.0,44.0,True,0,0,0
2,1,0,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,2,0,0
3,1,0,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,2,0,0
4,0,0,0,16.0,0,303.0,70.0,151.0,565.0,2.0,True,0,1,0


In [154]:
y=df['Transported']
X=df.drop(['Transported'],axis=1)

In [155]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2529)

In [156]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((6519, 13), (2174, 13), (6519,), (2174,))

In [157]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=472)
model.fit(X_train,y_train)

LogisticRegression(max_iter=472)

In [158]:
y_pred=model.predict(X_test)

In [159]:
model.score(X_test,y_test)

0.7920883164673413

In [160]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [161]:
accuracy_score(y_test,y_pred)

0.7920883164673413

In [162]:
confusion_matrix(y_test,y_pred)

array([[812, 241],
       [211, 910]])

In [163]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.79      0.77      0.78      1053
        True       0.79      0.81      0.80      1121

    accuracy                           0.79      2174
   macro avg       0.79      0.79      0.79      2174
weighted avg       0.79      0.79      0.79      2174



# **Testing Part**

In [164]:
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [165]:
test.Age.interpolate(inplace=True)
test.RoomService.interpolate(inplace=True)
test.FoodCourt.interpolate(inplace=True)
test.ShoppingMall.interpolate(inplace=True)
test.Spa.interpolate(inplace=True)
test.VRDeck.interpolate(inplace=True)
test.CryoSleep.fillna(True,inplace=True)
test.HomePlanet.fillna('Mars',inplace=True)
test.VIP.fillna(True,inplace=True)
test.Destination.fillna('PSO J318.5-22',inplace=True)

In [166]:
test[['deck', 'num', 'side']]=test.Cabin.str.split('/',expand=True)

In [167]:
test.deck.fillna('T',inplace=True)
test.num=test.num.astype(float)
test.num=test.num.interpolate(method='linear',limit_direction='both').round().astype(int)
test.side.fillna('P',inplace=True)

In [168]:
test.HomePlanet.replace({'Earth':0,'Europa':1,'Mars':2},inplace=True)

In [169]:
test.replace({'deck':{'F':0,'G':1,'E':2,'B':2,'C':2,'D':2,'A':2,'T':2}},inplace=True)

In [170]:
test.replace({'VIP':{False:0,True:1}},inplace=True)

In [171]:
test.replace({'Destination':{'TRAPPIST-1e':0,'55 Cancri e':1,'PSO J318.5-22':1}},inplace=True)

In [172]:
test.replace({'side':{'S':0,'P':1}},inplace=True)

In [173]:
test.replace({'CryoSleep':{False:0,True:1}},inplace=True)

In [174]:
test.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin           100
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name             94
deck              0
num               0
side              0
dtype: int64

In [175]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,deck,num,side
0,0013_01,0,1,G/3/S,0,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,1,3,0
1,0018_01,0,0,F/4/S,0,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0,4,0
2,0019_01,1,1,C/0/S,1,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,2,0,0
3,0021_01,1,0,C/1/S,0,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,2,1,0
4,0023_01,0,0,F/5/S,0,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0,5,0


In [176]:
df_test=test.drop(['PassengerId','Name','Cabin'],axis=1)

In [177]:
test['Transported']=model.predict(df_test)

In [178]:
test[['PassengerId','Transported']].to_csv('submission.csv',index=False)